In [1]:
import torch
import ttnn

torch.manual_seed(0)

device_id = 0
device = ttnn.open(device_id)

                  Metal | INFO     | Initializing device 0
                 Device | INFO     | Opening device driver
2023-10-31 20:13:05.409 | INFO     | SiliconDriver   - Detected 4 PCI devices
2023-10-31 20:13:05.431 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 3 device_id: 0xfaca revision: 0)
2023-10-31 20:13:05.437 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 2 device_id: 0xfaca revision: 0)
2023-10-31 20:13:05.441 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 1 device_id: 0xfaca revision: 0)
2023-10-31 20:13:05.450 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 0 device_id: 0xfaca revision: 0)
2023-10-31 20:13:05.555 | INFO     | SiliconDriver   - Disable PCIE DMA
2023-10-31 20:13:05.555 | INFO     | SiliconDriver   - Disable PCIE DMA
2023-10-31 20:13:05.555 | INFO  

# Enable program cache

In [2]:
ttnn.enable_program_cache()

                     Op | INFO     | Program Cache: enabled.


# Configuration

In [3]:
m = 1024
k = 1024
n = 1024

# Initialize tensors a and b with random values using torch

In [4]:
torch_a = torch.randn((m, k), dtype=torch.bfloat16)
torch_b = torch.randn((k, n), dtype=torch.bfloat16)

In [5]:
a = ttnn.from_torch(torch_a)
b = ttnn.from_torch(torch_b)

a = ttnn.to_device(a, device)
b = ttnn.to_device(b, device)

                     Op | INFO     | from_torch                                         external                                          
                     Op | INFO     | from_torch                                         external                                          
                     Op | INFO     | to_device                                          external                                          
                     Op | INFO     | to_device                                          external                                          


# Matrix multiply tensor a and b
The operation will run longer the first time because the kernels need to get compiled

In [6]:
%%timeit -n 1 -r 1
output = a @ b

                     Op | INFO     | tt::tt_metal::Tilize                               device                                            
773 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
                     Op | INFO     | tt::tt_metal::Tilize                               device                                            
                     Op | INFO     | tt::tt_metal::Matmul                               device                                            


In [7]:
%%timeit -n 2 -r 5
output = a @ b

                     Op | INFO     | tt::tt_metal::Tilize                               device                                            
299 µs ± 115 µs per loop (mean ± std. dev. of 5 runs, 2 loops each)
                     Op | INFO     | tt::tt_metal::Tilize                               device                                            
                     Op | INFO     | tt::tt_metal::Matmul                               device                                            
                     Op | INFO     | tt::tt_metal::Tilize                               device                                            
                     Op | INFO     | tt::tt_metal::Tilize                               device                                            
                     Op | INFO     | tt::tt_metal::Matmul                               device                                            
                     Op | INFO     | tt::tt_metal::Tilize                               device    

# Inspect the result of the matrix multiply

In [8]:
output = a @ b
output = ttnn.to_layout(output, ttnn.ROW_MAJOR_LAYOUT)

print("Printing ttnn tensor")
print(output.shape)
print(output[:1, :32])

                     Op | INFO     | tt::tt_metal::Tilize                               device                                            
Printing ttnn tensor
[1024, 1024]
                     Op | INFO     | tt::tt_metal::Tilize                               device                                            
                     Op | INFO     | tt::tt_metal::Matmul                               device                                            
                     Op | INFO     | tt::tt_metal::Untilize                             device                                            
                     Op | INFO     | from_device                                        external                                          
                     Op | INFO     | to_torch                                           external                                          
Tensor([ [34.25, 9.625, 11.3125, 0.964844, 1.45312, -26.875, 23.125, -1.39062, -20.375, 33, 5.8125, 10.6875, -18.625, 14.5, -42.75, 

# Tilize tensors before running matrix multiply

In [9]:
a = ttnn.to_layout(a, ttnn.TILE_LAYOUT)
b = ttnn.to_layout(b, ttnn.TILE_LAYOUT)

                     Op | INFO     | tt::tt_metal::Tilize                               device                                            
                     Op | INFO     | tt::tt_metal::Tilize                               device                                            


The operation will execute much faster because it will re-use compiled kernels

In [10]:
%%timeit -n 2 -r 5
output = a @ b

106 µs ± 24.3 µs per loop (mean ± std. dev. of 5 runs, 2 loops each)
                     Op | INFO     | tt::tt_metal::Matmul                               device                                            
                     Op | INFO     | tt::tt_metal::Matmul                               device                                            
                     Op | INFO     | tt::tt_metal::Matmul                               device                                            
                     Op | INFO     | tt::tt_metal::Matmul                               device                                            
                     Op | INFO     | tt::tt_metal::Matmul                               device                                            
                     Op | INFO     | tt::tt_metal::Matmul                               device                                            
                     Op | INFO     | tt::tt_metal::Matmul                               device   

# Matrix multiply tensor a and b by using more performant config

In [11]:
a = ttnn.from_torch(torch_a)
b = ttnn.from_torch(torch_b)

a = ttnn.to_device(a, device, memory_config=ttnn.L1_MEMORY_CONFIG)
b = ttnn.to_device(b, device, memory_config=ttnn.L1_MEMORY_CONFIG)

a = ttnn.to_layout(a, ttnn.TILE_LAYOUT)
b = ttnn.to_layout(b, ttnn.TILE_LAYOUT)

                     Op | INFO     | from_torch                                         external                                          
                     Op | INFO     | from_torch                                         external                                          
                     Op | INFO     | to_device                                          external                                          
                     Op | INFO     | to_device                                          external                                          
                     Op | INFO     | tt::tt_metal::Tilize                               device                                            
                     Op | INFO     | tt::tt_metal::Tilize                               device                                            


In [12]:
%%timeit -n 1 -r 1
output = ttnn.matmul(a, b, memory_config=ttnn.L1_MEMORY_CONFIG, core_grid=(8, 8))

                     Op | INFO     | tt::operations::primary::Matmul                    device                                            
387 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [13]:
%%timeit -n 2 -r 5
output = ttnn.matmul(a, b, memory_config=ttnn.L1_MEMORY_CONFIG, core_grid=(8, 8))

221 µs ± 57 µs per loop (mean ± std. dev. of 5 runs, 2 loops each)
                     Op | INFO     | tt::operations::primary::Matmul                    device                                            
                     Op | INFO     | tt::operations::primary::Matmul                    device                                            
                     Op | INFO     | tt::operations::primary::Matmul                    device                                            
                     Op | INFO     | tt::operations::primary::Matmul                    device                                            
                     Op | INFO     | tt::operations::primary::Matmul                    device                                            
                     Op | INFO     | tt::operations::primary::Matmul                    device                                            
                     Op | INFO     | tt::operations::primary::Matmul                    device     

# Close the device

In [14]:
ttnn.close(device)

                  Metal | INFO     | Closing device 0
